In [103]:
from sendgrid.helpers.mail import Mail, Email, To, Content
import sendgrid
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, output_guardrail, GuardrailFunctionOutput
from pypdf import PdfReader
from typing import Dict
from openai import AsyncOpenAI
from pydantic import BaseModel

In [104]:
load_dotenv(override=True)

True

In [105]:
@function_tool
def read_resume():
    """ Read client's resume"""
    resume_reader = PdfReader("me/Jongkook Kim - Resume.pdf")
    resume = ""
    for page in resume_reader.pages:
        page_text = page.extract_text()
        if page_text:
            resume += page_text.replace('\t', ' ').replace('\n', ' ')

    return resume

In [106]:
resume_summary_instruction = """You are a head hunter agent working for a client who is looking a job.
Your client is a senior level sotware engineer and looking for principle or director level poisition.
You summarize a client's resume includes technical skills, resposibilities and accomplishments.
Summarized resume must compily follow requriements.
1. Modern technical skills based on current job market and IT trends in year 2025.
2. Qualfied for director or principle software engeering role.
3. Summarized content should be 3 ~ 4 paragraphes

##Resume\n{resume}\n
"""

resume_summarizor = Agent(
    name="Resume Summarizor",
    instructions=resume_summary_instruction,
    model="gpt-4o-mini",
    tools=[read_resume]
)

resume_summarize_tool = resume_summarizor.as_tool(
    tool_name="resume_summarizor",
    tool_description="Summarize client resume to apeal HR or hiring manager"
)

In [107]:
resume_summary_evaluator_instruction = """
You are a head hunter agent working for a client who is looking a job. 
Your role is evaluating summarized client's resume to meet following requriements.
1. Modern technical skills based on current job market and IT trends in year 2025.
2. Qualfied for director or principle software engeering role.
3. Summarized content should be 3 ~ 4 paragraphes

If the summarized resume doesn't meet the requrements, give feedback why it fails the evaluation, 
so resume_summarize_tool can summarize again using your feedback.
"""

resume_summary_evaluator = Agent(
    name="Summarized Resume Evaluator",
    instructions=resume_summary_evaluator_instruction,
    tools=[read_resume],
    model="gpt-4o-mini"
)

resume_summary_evaluator_tool = resume_summary_evaluator.as_tool(
    tool_name="resume_summary_evaluator",
    tool_description="Evaluate summerized resume"
)

In [108]:

resume_summary_manager_instruction = """
You are a coordinator who works between resume_summarizor tool and resume_summary_evaluator tool.
Your goal is get the best client resume summary version.


To acomplish your goal, you need to follow below steps.
1. resume_summarizor tool
1-1. call this tool to get summarized resume. 
1-2. If resume_summarizor is called less than 3 times, call resume_summarizor tool. When you call resume_summarizor, pass the summarized resume, so resume_summarizor tool can evaluaate it.
1-3. If resume_summarizor tool has been invoked 3 times, hands off the latest summary resume to email_sendor agant.

2. resume_summarizor tool
2-1. Call this tool after resume_summarizor tool completes summarizion. When you call resume_summarizor agent, pass the response of resume_summarizor agent to resume_summarizor agent

Crucial Rules:
- Make sure resume_summarizor tool is not invoked more than 2 times.
- Consider the latest summary resume as a best version.
"""

resume_summary_manager = Agent(
    name="Resume Summary Coordinator",
    instructions=resume_summary_manager_instruction,
    model="gpt-4o-mini",
    tools=[resume_summarize_tool, resume_summary_evaluator_tool]
)

resume_summary_manager_tool = resume_summary_manager.as_tool(
    tool_name="resume_summary_manager_tool",
    tool_description="coordinate summary and evaluation resume"
)

In [109]:
class PIICheckOutput(BaseModel):
    is_pii_in_message: bool
    pii: str

pii_check_instruction = """
You are a compliance officer who verifies digital content and identify client's PII information is in it.
Check if the email body including PII.
Examples of PII are including, but not limited to name, address, email address, phone number.
"""

pii_guardrail_agent = Agent(
    name="PII Check",
    instructions=pii_check_instruction,
    output_type=PIICheckOutput,
    model="gpt-4o-mini"
)

@output_guardrail
async def guardrail_against_pii(ctx, agent, message):
    result = await Runner.run(pii_guardrail_agent, message, context=ctx.context)
    is_pii_in_message = result.final_output.is_pii_in_message
    return GuardrailFunctionOutput(
        output_info={"found_pii":result.final_output},
        tripwire_triggered=is_pii_in_message)

In [110]:
@function_tool
def send_email(subject: str, body: str) -> Dict[str, str]:
    """ Send out an email with given subject and body to all HR or hiring managers"""
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("gaulbi@gmail.com")
    to_email = To("gaulbi@gmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email=from_email, to_emails=to_email, subject=subject, plain_text_content=content).get()
    response = sg.client.mail.send.post(request_body=mail)
    
    return {"status": response.status_code}

create_email_instruction = """You are a head hunter agent working for a client who is looking a job.
Your client is a senior level sotware engineer and looking for principle or director level poisition.
You write professional, serious cold emails based on summarized client's resume.
When you write email, do not include any client PII (including, but not limited to name, address, email address, phone number.).
Explanation of the client professional experience in email should be accurate and appeals to HR or hiring manager. 

##Resume\n{resume}\n
"""

email_sendor = Agent(
    name="Email Generator",
    instructions=create_email_instruction,
    model="gpt-4o-mini",
    tools=[send_email],
    output_guardrails=[guardrail_against_pii]
)

In [111]:
head_hunter_manager_instruction = """
You are a Job Head Hunter Manager. 

Follow below steps carfully.
1. call resume_summary_manager agent to get the best summarized resume version. Call resume_summary_manager tool only one time.
2. When step 1 is completed, generate summary bsaed on the summarized resume
3. when step 2 is completed, call email_sendor agent and pass the subject and summaried resume
"""

head_hunter_manager = Agent(
    name="Head Hunter Manager",
    instructions=head_hunter_manager_instruction,
    tools=[resume_summary_manager_tool, read_resume],
    model="gpt-4o-mini",
    handoffs=[email_sendor]
)

In [112]:
message = "Summarize client resume and send email "
with trace("Automated Job Head Hunter"):
    result = await Runner.run(head_hunter_manager, message)

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 92 0 (offset 0)
Ignoring wrong pointing object 93 0 (offset 0)
